In [32]:
# ============================================
# ILD(ИЗЛ) vs COPD(ХОБЛ): непараметрический анализ в Colab
# - загрузка файла из диалога/Drive
# - long/wide автоопределение
# - Wilcoxon (парные) внутри групп по парам лет
# - Mann–Whitney между группами (по годам и по дельтам)
# - FDR(BH) отдельно для within/between, эффекты (rank-biserial, Cliff’s δ)
# - экспорт Excel/CSV + PNG-графики, скачивание ZIP
# ============================================

# !pip -q install pandas numpy scipy statsmodels matplotlib openpyxl python-slugify

import os, re, glob, warnings, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats
from statsmodels.stats.multitest import multipletests
from slugify import slugify

warnings.filterwarnings("ignore")

# ---------- НАСТРОЙКИ ----------
YEARS      = ["2023", "2024", "2025"]   # ожидаемые года (будут авто-детектиться при long)
MIN_PAIRED = 5                          # минимум пар для Wilcoxon
AUTO_DOWNLOAD = True                    # автоматически скачать ZIP с результатами

# Если хотите руками задать путь (например из Google Drive) — раскомментируйте:
# INFILE = "/content/drive/MyDrive/папка/pulmo_polarity_aligned итоговый файл.xlsx"

# ---------- УТИЛИТЫ ----------
def clean_names(cols):
    out=[]
    for c in cols:
        c = str(c)
        c = re.sub(r"[\u00A0\u2009]", " ", c).strip()
        c = re.sub(r"\s+", " ", c)
        out.append(c)
    return out

def map_group(x):
    s = str(x).strip()
    if s in {"1","ИЛД","ИЗЛ","ILD","ild","Izl","IZL"}: return "ИЗЛ"
    if s in {"2","ХОБЛ","COPD","copd","Copd","Хобл"}: return "ХОБЛ"
    return np.nan

def read_any(path, sheet=0):
    ext = Path(path).suffix.lower()
    if ext in [".xlsx",".xls"]:
        return pd.read_excel(path, sheet_name=sheet)
    elif ext == ".csv":
        try:
            return pd.read_csv(path, encoding="utf-8")
        except Exception:
            return pd.read_csv(path, encoding="cp1251", sep=";")
    else:
        raise ValueError("Поддерживаются .xlsx/.xls/.csv")

def find_col(df, cands):
    low = [c.lower() for c in df.columns]
    for cand in cands:
        if cand.lower() in low:
            return df.columns[low.index(cand.lower())]
    return None

def cliffs_delta(x, y):
    x = np.asarray(x, float); y = np.asarray(y, float)
    x = x[~np.isnan(x)]; y = y[~np.isnan(y)]
    if len(x)==0 or len(y)==0: return np.nan
    gt = sum((xi > y).sum() for xi in x)
    lt = sum((xi < y).sum() for xi in x)
    return (gt - lt) / (len(x)*len(y))

def paired_rank_biserial(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float)
    d = b - a
    pos = np.sum(d > 0); neg = np.sum(d < 0)
    nz  = pos + neg
    return 0.0 if nz == 0 else (pos - neg) / nz

def add_fdr(df):
    if df is None or df.empty:
        return df
    q = multipletests(df["p_raw"].values, method="fdr_bh")[1]
    out = df.copy()
    out["q_fdr_bh"] = q
    return out

# ---------- ЗАГРУЗКА ФАЙЛА (диалог или Drive) ----------
try:
    INFILE
except NameError:
    INFILE = None

if INFILE is None:
    print("⬆️ Выберите Excel/CSV файл для загрузки…")
    try:
        from google.colab import files
        uploaded = files.upload()
        if uploaded:
            INFILE = next(iter(uploaded.keys()))
            print("✅ Загружен файл:", INFILE)
    except Exception as e:
        print("⚠️ Не в Colab или диалог не сработал:", e)

if INFILE is None:
    # Попытка найти оригинальный файл по части имени
    cand = [p for p in os.listdir(".") if p.lower().endswith((".xlsx",".xls",".csv"))]
    if cand:
        INFILE = cand[0]
        print("ℹ️ Использую найденный файл:", INFILE)

assert INFILE is not None and Path(INFILE).exists(), f"Файл не найден: {INFILE}"

# ---------- ЧТЕНИЕ И ПРИВЕДЕНИЕ К LONG ----------
SHEET_NAME = 0  # при необходимости замените на имя листа
df0 = read_any(INFILE, SHEET_NAME)
df0.columns = clean_names(df0.columns)

ID_CANDS    = ["id","patient_id","пациент","№","ид","код","uid"]
GROUP_CANDS = ["group","группа","диагноз"]
YEAR_CANDS  = ["year","год","time","время"]
MARK_CANDS  = ["marker","показатель","параметр","анализ","variable","feature","маркер"]
VAL_CANDS   = ["value","значение","val","y"]

id_col   = find_col(df0, ID_CANDS)
grp_col  = find_col(df0, GROUP_CANDS)
year_col = find_col(df0, YEAR_CANDS)
mark_col = find_col(df0, MARK_CANDS)
val_col  = find_col(df0, VAL_CANDS)

is_long = all(c is not None for c in [id_col, grp_col, year_col, mark_col, val_col])

if is_long:
    print("✅ Обнаружен длинный формат.")
    dat = df0.rename(columns={id_col:"id", grp_col:"group", year_col:"year",
                              mark_col:"marker", val_col:"value"}).copy()
    # авто-детект лет, если в данных есть что-то отличное от YEARS
    years_detected = sorted(list(map(str, pd.Series(dat["year"]).astype(str).unique())))
    if set(years_detected).issuperset(set(YEARS)) or len(years_detected) <= len(YEARS):
        YEARS = [y for y in YEARS if y in years_detected] or years_detected
else:
    print("ℹ️ Похоже на широкий формат. Преобразую по суффиксам _ГОД.")
    # возьмём возможные id/group
    id_vals  = df0[id_col].astype(str) if id_col else pd.Series(range(1, len(df0)+1), index=df0.index, dtype=str)
    grp_vals = df0[grp_col] if grp_col else pd.Series([np.nan]*len(df0))
    # regex: базовое имя + год в конце
    rx = re.compile(r"^(.*?)[\._\s]?(\d{4})$")
    rows=[]
    years_found = set()
    for col in df0.columns:
        m = rx.match(col)
        if not m:
            continue
        base, year = m.group(1).strip(), m.group(2)
        years_found.add(year)
        vals = pd.to_numeric(df0[col], errors="coerce")
        rows.append(pd.DataFrame({
            "id": id_vals.values,
            "group": grp_vals.values,
            "year": year,
            "marker": base or col,
            "value": vals
        }))
    if not rows:
        raise RuntimeError("Не нашёл ни одного столбца с годами (…_2023/… 2024/…2025). Проверьте имена.")
    YEARS = sorted(list(years_found))
    dat = pd.concat(rows, ignore_index=True)

# приведение типов, отчистка
dat["group"]  = dat["group"].map(map_group)
dat["year"]   = dat["year"].astype(str).str.strip()
dat["marker"] = dat["marker"].astype(str)
dat = dat.dropna(subset=["value","marker"])
dat = dat[dat["year"].isin(YEARS)]
dat["id"] = dat["id"].astype(str)

# на случай дублей: агрегируем по (id, group, marker, year) медианой
dat = (dat.groupby(["id","group","marker","year"], as_index=False)["value"]
          .median())

print(f"Маркеров: {dat['marker'].nunique()}, лет: {YEARS}, записей: {len(dat)}")

# ---------- СВОДНЫЕ МЕДИАНЫ ----------
summary_medians = (
    dat.groupby(["marker","group","year"], dropna=False)
       .agg(N=("value","size"),
            median=("value","median"),
            Q1=("value", lambda x: np.quantile(x, 0.25)),
            Q3=("value", lambda x: np.quantile(x, 0.75)))
       .reset_index()
       .sort_values(["marker","group","year"])
)

# ---------- ВНУТРИГРУППОВЫЕ: Wilcoxon (парные) ----------
pairs = []
if "2023" in YEARS and "2024" in YEARS: pairs.append(("2024","2023"))
if "2024" in YEARS and "2025" in YEARS: pairs.append(("2025","2024"))
if "2023" in YEARS and "2025" in YEARS: pairs.append(("2025","2023"))

within_rows = []
for mk in dat["marker"].unique():
    for gr in ["ИЗЛ","ХОБЛ"]:
        d = dat[(dat["marker"]==mk) & (dat["group"]==gr)]
        for y2, y1 in pairs:
            w = (d[d["year"]==y1][["id","value"]]
                 .merge(d[d["year"]==y2][["id","value"]], on="id", suffixes=(f"_{y1}", f"_{y2}"))
                 .dropna())
            if len(w) >= MIN_PAIRED:
                a = w[f"value_{y1}"].to_numpy()
                b = w[f"value_{y2}"].to_numpy()
                try:
                    p = stats.wilcoxon(b, a, zero_method="wilcox", alternative="two-sided", method="auto").pvalue
                except Exception:
                    p = stats.wilcoxon(b, a, zero_method="wilcox", alternative="two-sided").pvalue
                rb = paired_rank_biserial(a, b)
                delta = b - a
                within_rows.append(dict(
                    family="within", marker=mk, group=gr, period=f"Δ({y2}–{y1})",
                    N_pairs=len(w),
                    median_change=float(np.median(delta)),
                    Q1_change=float(np.quantile(delta, 0.25)),
                    Q3_change=float(np.quantile(delta, 0.75)),
                    p_raw=float(p), effect=float(rb), effect_type="rank-biserial"
                ))
within_tbl = pd.DataFrame(within_rows)

# ---------- МЕЖГРУППОВЫЕ: Mann–Whitney (по годам и по Δ) ----------
between_rows = []

# по годам
for mk in dat["marker"].unique():
    for yr in YEARS:
        d_yr = dat[(dat["marker"]==mk) & (dat["year"]==yr)]
        x = d_yr.loc[d_yr["group"]=="ИЗЛ","value"].to_numpy()
        y = d_yr.loc[d_yr["group"]=="ХОБЛ","value"].to_numpy()
        if len(x) >= 5 and len(y) >= 5:
            p = stats.mannwhitneyu(x, y, alternative="two-sided").pvalue
            cd = cliffs_delta(x, y)
            between_rows.append(dict(
                family="between", marker=mk, contrast="ИЛД_vs_ХОБЛ",
                period=yr, N_ILD=int(len(x)), N_COPD=int(len(y)),
                p_raw=float(p), effect=float(cd), effect_type="Cliff_delta"
            ))

# по дельтам (устойчиво к отсутствию годов)
for mk in dat["marker"].unique():
    d_m = dat[dat["marker"]==mk]
    for y2, y1 in pairs:
        ild = (d_m[(d_m["group"]=="ИЗЛ") & (d_m["year"].isin([y1, y2]))]
               .pivot_table(index="id", columns="year", values="value", aggfunc="first"))
        copd = (d_m[(d_m["group"]=="ХОБЛ") & (d_m["year"].isin([y1, y2]))]
                .pivot_table(index="id", columns="year", values="value", aggfunc="first"))
        for df_ in (ild, copd):
            df_.columns = df_.columns.astype(str)
            for col in [y1, y2]:
                if col not in df_.columns: df_[col] = np.nan
        ild = ild.dropna(subset=[y1, y2], how="any")
        copd = copd.dropna(subset=[y1, y2], how="any")
        if len(ild) >= 5 and len(copd) >= 5:
            di = (ild[y2] - ild[y1]).to_numpy()
            dc = (copd[y2] - copd[y1]).to_numpy()
            p  = stats.mannwhitneyu(di, dc, alternative="two-sided").pvalue
            cd = cliffs_delta(di, dc)
            between_rows.append(dict(
                family="between", marker=mk, contrast="Δ(ИЛД)_vs_Δ(ХОБЛ)",
                period=f"Δ({y2}–{y1})", N_ILD=int(len(di)), N_COPD=int(len(dc)),
                p_raw=float(p), effect=float(cd), effect_type="Cliff_delta"
            ))

between_tbl = pd.DataFrame(between_rows)

# ---------- FDR(BH) ОТДЕЛЬНО ----------
within_tbl  = add_fdr(within_tbl)
between_tbl = add_fdr(between_tbl)

# ---------- ЭКСПОРТ ----------
Path("outputs").mkdir(exist_ok=True)

summary_out = summary_medians.copy()
summary_out.to_csv("outputs/medians_by_group_year.csv", index=False)

if within_tbl is not None and not within_tbl.empty:
    within_out = within_tbl[["marker","group","period","N_pairs",
                             "median_change","Q1_change","Q3_change",
                             "p_raw","q_fdr_bh","effect","effect_type"]] \
                             .sort_values(["marker","group","period"])
    within_out.to_csv("outputs/within_results.csv", index=False)
else:
    within_out = pd.DataFrame()

if between_tbl is not None and not between_tbl.empty:
    between_out = between_tbl[["marker","contrast","period","N_ILD","N_COPD",
                               "p_raw","q_fdr_bh","effect","effect_type"]] \
                               .sort_values(["marker","period","contrast"])
    between_out.to_csv("outputs/between_results.csv", index=False)
else:
    between_out = pd.DataFrame()

with pd.ExcelWriter("outputs/results_all.xlsx", engine="openpyxl") as xlw:
    summary_out.to_excel(xlw, sheet_name="medians", index=False)
    if not within_out.empty:  within_out.to_excel(xlw, sheet_name="within",  index=False)
    if not between_out.empty: between_out.to_excel(xlw, sheet_name="between", index=False)

print("\n✅ Готово.")
print(f"Всего маркеров: {dat['marker'].nunique()}")
print(f"Внутригрупповых сравнений: {0 if within_tbl is None else len(within_tbl)}")
print(f"Межгрупповых сравнений:    {0 if between_tbl is None else len(between_tbl)}")
print("Файлы: outputs/results_all.xlsx, outputs/*.csv")

# ---------- ГРАФИКИ ----------
Path("plots").mkdir(exist_ok=True)

# звёздочки межгрупповой значимости по годам (после FDR)
if between_tbl is not None and not between_tbl.empty:
    sig_year = (between_tbl
                .loc[(between_tbl["contrast"]=="ИЛД_vs_ХОБЛ") & (between_tbl["period"].isin(YEARS)),
                     ["marker","period","q_fdr_bh"]]
                .rename(columns={"period":"year"}))
    def star(q):
        return "***" if q < 1e-3 else ("**" if q < 1e-2 else ("*" if q < 0.05 else ""))
    sig_year["star"] = sig_year["q_fdr_bh"].map(star)
else:
    sig_year = pd.DataFrame(columns=["marker","year","q_fdr_bh","star"])

for mk in dat["marker"].unique():
    d = dat[dat["marker"]==mk]
    if d.empty:
        continue

    fig = plt.figure(figsize=(10,6))
    ax = plt.gca()

    xpos = np.arange(len(YEARS))
    width = 0.38

    for i, yr in enumerate(YEARS):
        for j, gr in enumerate(["ИЗЛ","ХОБЛ"]):
            vals = d[(d["year"]==yr) & (d["group"]==gr)]["value"].dropna().to_numpy()
            if len(vals)==0:
                continue
            x = i + (j-0.5)*width
            bp = ax.boxplot(vals, positions=[x], widths=width*0.9, patch_artist=True, manage_ticks=False)
            ax.scatter(np.full_like(vals, x, dtype=float), vals, s=6, alpha=0.35)

    ax.set_xticks(np.arange(len(YEARS)))
    ax.set_xticklabels(YEARS)
    ax.set_title(mk)
    ax.set_xlabel("")
    ax.set_ylabel("Значение")
    ax.legend(["ИЗЛ","ХОБЛ"], loc="upper right")

    # звёздочки
    ann = sig_year[sig_year["marker"]==mk]
    if not ann.empty:
        y_max = d.groupby("year")["value"].max().reindex(YEARS)
        y_span = (np.nanmax(d["value"]) - np.nanmin(d["value"])) or 1.0
        for _, row in ann.iterrows():
            if row["year"] in y_max.index and not pd.isna(y_max.loc[row["year"]]):
                ax.text(YEARS.index(row["year"]), y_max.loc[row["year"]] + 0.05*y_span,
                        row["star"], ha="center", va="bottom", fontsize=14)

    plt.tight_layout()
    fname = f"plots/box_{slugify(mk)}.png"
    plt.savefig(fname, dpi=140)
    plt.close()

print("🖼️ Графики сохранены в папке plots/")

# ---------- СКАЧАТЬ ZIP ----------
zip_path = "/content/ILD_COPD_outputs.zip"
if os.path.exists(zip_path):
    os.remove(zip_path)
shutil.make_archive("/content/ILD_COPD_outputs", "zip", "/content")
print("ZIP:", zip_path)

if AUTO_DOWNLOAD:
    try:
        from google.colab import files
        files.download(zip_path)
    except Exception as e:
        print("ℹ️ Не удалось авто-скачать:", e, "\nСкачайте ZIP через панель Files слева.")


ℹ️ Похоже на широкий формат. Преобразую по суффиксам _ГОД.
Маркеров: 0, лет: ['2023', '2024', '2025'], записей: 0

✅ Готово.
Всего маркеров: 0
Внутригрупповых сравнений: 0
Межгрупповых сравнений:    0
Файлы: outputs/results_all.xlsx, outputs/*.csv
🖼️ Графики сохранены в папке plots/


KeyboardInterrupt: 

In [35]:
# -*- coding: utf-8 -*-
# Colab script: Pulmo profile page (medians/IQR, within/between p, FDR)
!pip -q install openpyxl statsmodels scipy pandas

import re
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu, wilcoxon
from statsmodels.stats.multitest import multipletests

# ==== 1) ЗАГРУЗКА ФАЙЛА ====
try:
    from google.colab import files  # type: ignore
    uploaded = files.upload()       # выберите ваш .xlsx
    infile = next(iter(uploaded.keys()))
except Exception:
    # Если запускаете не в Colab — укажите путь вручную
    infile = "/content/файл_для_расчета.xlsx"

df = pd.read_excel(infile)

# ==== 2) НАСТРОЙКИ ====
GROUP_COL_CANDS = ["диагноз", "диагнoз", "group", "Группа", "группа"]
group_col = next((c for c in GROUP_COL_CANDS if c in df.columns), None)
if group_col is None:
    raise RuntimeError("Не нашёл колонку с группой. Добавьте столбец 'диагноз' (1=ИЗЛ, 2=ХОБЛ).")

GROUP_LABELS = {1: "ИЗЛ", 2: "ХОБЛ"}   # при необходимости: {1: "ИЗЛ-ССД", 2: "ХОБЛ"}

DECIMALS = 3  # сколько знаков показывать в МЕ/Q1/Q3 и p/q

# ==== 3) ПОИСК МАРКЕРОВ ПУЛЬМОПРОФИЛЯ ====
year_re = re.compile(r"_(\d{4})$")
marker_cols = [c for c in df.columns if year_re.search(str(c))]
if not marker_cols:
    raise RuntimeError("Не нашёл столбцов вида '<показатель>_ГОД'.")

all_markers = sorted(set(re.sub(year_re, "", c) for c in marker_cols))
years = sorted(set(int(year_re.search(c).group(1)) for c in marker_cols))

# Нормализация для робких сопоставлений (латиница↔кириллица и удаление '_'/' ')
def norm_text(s: str) -> str:
    mapping = {
        "С":"C","с":"c","О":"O","о":"o","Т":"T","т":"t","М":"M","м":"m",
        "Х":"X","х":"x","В":"B","в":"b","Р":"P","р":"p","Н":"H","н":"h",
        "К":"K","к":"k","А":"A","а":"a","Е":"E","е":"e","У":"Y","у":"y",
        "Ё":"E","ё":"e","Й":"I","й":"i","Л":"L","л":"l","Д":"D","д":"d"
    }
    s2 = "".join(mapping.get(ch, ch) for ch in str(s))
    s2 = s2.lower().replace("_", "").replace(" ", "")
    return s2

# Ключи пульмопрофиля (после norm_text)
pulmo_keys = [
    # 6-минутная ходьба, ЧД (до/после), разница
    "дистанцият6мх","t6мх","t6mx","т6мх",
    "чдвминуту","разницачддт6мх",
    # SaO2 и Борг
    "sao2pct","sao2","saо2pct","шкалаборга",
    # спирометрия
    "фжел","офв1","oфв1","ofv1",
    # газы и лактат выдоха/крови
    "co2ввыдыхаемомвоздухе","ph","po2","pco2","lac",
    # КТ и объёмы
    "объемлегких","обьемлегких",
    "денситометричесkieпоказатели","денситометрическиепоказатели","денситометрическиепоказатели",
    "плотностьфиброза","плотностьфиброза"
]

pulmo_markers = [m for m in all_markers if any(k in norm_text(m) for k in pulmo_keys)]
if not pulmo_markers:  # на всякий случай — если список пустой, берём все
    pulmo_markers = all_markers.copy()

# ==== 4) LONG-ФОРМАТ ====
rows_long = []
for m in pulmo_markers:
    for y in years:
        col = f"{m}_{y}"
        if col in df.columns:
            vals = pd.to_numeric(df[col], errors="coerce")
            rows_long.append(pd.DataFrame({
                "marker": m,
                "year": y,
                "group": df[group_col].map(GROUP_LABELS).fillna(df[group_col].astype(str)),
                "value": vals
            }))
long_df = pd.concat(rows_long, ignore_index=True)

# ==== 5) ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ====
def med_q1_q3(x):
    x = pd.to_numeric(pd.Series(x), errors="coerce").dropna()
    if len(x) == 0:
        return np.nan, np.nan, np.nan, 0
    return (float(np.nanmedian(x)),
            float(np.nanpercentile(x, 25)),
            float(np.nanpercentile(x, 75)),
            int(x.notna().sum()))

def fmt_num(v):
    return "NA" if pd.isna(v) else f"{v:.{DECIMALS}g}"

def fmt_stat(med, q1, q3, n):
    return f"{fmt_num(med)} [{fmt_num(q1)}; {fmt_num(q3)}] (n={n})"

# ==== 6) МЕЖГРУППОВЫЕ P ПО ГОДАМ (Mann–Whitney) + FDR ====
between_rows = []
for m in pulmo_markers:
    for y in years:
        sub = long_df[(long_df["marker"] == m) & (long_df["year"] == y)]
        v_ild  = sub[sub["group"] == "ИЗЛ"]["value"].dropna().values
        v_copd = sub[sub["group"] == "ХОБЛ"]["value"].dropna().values
        if len(v_ild) > 0 and len(v_copd) > 0:
            try:
                _, p = mannwhitneyu(v_ild, v_copd, alternative="two-sided")
            except ValueError:
                p = np.nan
        else:
            p = np.nan
        between_rows.append({"marker": m, "year": y, "p_between": p})

between_df = pd.DataFrame(between_rows)
if not between_df.empty:
    mask = between_df["p_between"].notna()
    q = np.full(len(between_df), np.nan)
    if mask.any():
        _, qvals, _, _ = multipletests(between_df.loc[mask, "p_between"], method="fdr_bh")
        q[mask.values] = qvals
    between_df["q_between"] = q

# ==== 7) ВНУТРИГРУППОВЫЕ P (Wilcoxon) ТОЛЬКО СОСЕДНИЕ ГОДЫ + FDR ====
year_pairs = {years[i+1]: years[i] for i in range(len(years)-1)}  # {2024:2023, 2025:2024, ...}
within_rows = []
for m in pulmo_markers:
    for g_code, g_name in GROUP_LABELS.items():
        sub_idx = df[group_col] == g_code
        wide = {}
        for y in years:
            col = f"{m}_{y}"
            if col in df.columns:
                wide[y] = pd.to_numeric(df.loc[sub_idx, col], errors="coerce").reset_index(drop=True)
        if not wide:
            continue
        wide_df = pd.DataFrame(wide)
        for y2, y1 in year_pairs.items():
            if y1 in wide_df and y2 in wide_df:
                v1, v2 = wide_df[y1], wide_df[y2]
                mask = v1.notna() & v2.notna()
                if mask.sum() > 1:
                    try:
                        _, p = wilcoxon(v1[mask], v2[mask], alternative="two-sided", zero_method="wilcox")
                    except ValueError:
                        p = np.nan
                else:
                    p = np.nan
                within_rows.append({"marker": m, "group": g_name, "year": y2, "prev_year": y1, "p_within": p})

within_df = pd.DataFrame(within_rows)
if not within_df.empty:
    mask = within_df["p_within"].notna()
    q = np.full(len(within_df), np.nan)
    if mask.any():
        _, qvals, _, _ = multipletests(within_df.loc[mask, "p_within"], method="fdr_bh")
        q[mask.values] = qvals
    within_df["q_within"] = q

# ==== 8) СБОРКА «СТРАНИЦЫ» ПУЛЬМОПРОФИЛЯ ====
label_map = {y: (["a)","b)","c)","d)"][i] if i < 4 else f"{i+1})") for i, y in enumerate(years)}

rows = []
for m in pulmo_markers:
    for y in years:
        sub = long_df[(long_df["marker"] == m) & (long_df["year"] == y)]
        med_i, q1_i, q3_i, n_i = med_q1_q3(sub[sub["group"] == "ИЗЛ"]["value"])
        med_c, q1_c, q3_c, n_c = med_q1_q3(sub[sub["group"] == "ХОБЛ"]["value"])

        rb = between_df[(between_df["marker"] == m) & (between_df["year"] == y)]
        p_b = rb["p_between"].iloc[0] if not rb.empty else np.nan
        q_b = rb["q_between"].iloc[0] if not rb.empty else np.nan

        rw_i = within_df[(within_df["marker"] == m) & (within_df["group"] == "ИЗЛ") & (within_df["year"] == y)]
        p_w_i = rw_i["p_within"].iloc[0] if not rw_i.empty else np.nan
        q_w_i = rw_i["q_within"].iloc[0] if not rw_i.empty else np.nan

        rw_c = within_df[(within_df["marker"] == m) & (within_df["group"] == "ХОБЛ") & (within_df["year"] == y)]
        p_w_c = rw_c["p_within"].iloc[0] if not rw_c.empty else np.nan
        q_w_c = rw_c["q_within"].iloc[0] if not rw_c.empty else np.nan

        rows.append({
            "Variables": m,
            "строка": f"{label_map[y]} {y} год",
            "МЕ [Q1;Q3] ИЗЛ": fmt_stat(med_i, q1_i, q3_i, n_i),
            "Р внутригрупповой (ИЗЛ)": (np.nan if y == years[0] else p_w_i),
            "Р скорректированный (ИЗЛ)": (np.nan if y == years[0] else q_w_i),
            "МЕ [Q1;Q3] ХОБЛ": fmt_stat(med_c, q1_c, q3_c, n_c),
            "Р внутригрупповой (ХОБЛ)": (np.nan if y == years[0] else p_w_c),
            "Р скорректированный (ХОБЛ)": (np.nan if y == years[0] else q_w_c),
            "Р межгрупповой": p_b,
            "Р скорректированный (межгрупповой)": q_b
        })

pulmo_page = pd.DataFrame(rows)
pulmo_page["year"] = pulmo_page["строка"].str.extract(r"(\d{4})").astype(int)
pulmo_page = pulmo_page.sort_values(["Variables", "year"]).drop(columns=["year"]).reset_index(drop=True)

# ==== 9) СОХРАНЕНИЕ ====
out_xlsx = "pulmo_profile_page.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as wr:
    pulmo_page.to_excel(wr, sheet_name="pulmo_profile", index=False)
    within_df.to_excel(wr, sheet_name="within_raw", index=False)
    between_df.to_excel(wr, sheet_name="between_raw", index=False)

print("Saved:", out_xlsx)
try:
    from google.colab import files  # type: ignore
    files.download(out_xlsx)
except Exception:
    pass


Saving файл для рассчета.xlsx to файл для рассчета.xlsx
Saved: pulmo_profile_page.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
# -*- coding: utf-8 -*-
# Pulmo profile → formatted Excel + Word (с расширенным подбором показателей)
!pip -q install pandas scipy statsmodels openpyxl python-docx
out_xlsx = "pulmo_profile_formatted.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    pulmo_page.to_excel(writer, sheet_name="pulmo_profile", index=False)
    within_df.to_excel(writer, sheet_name="within_raw", index=False)
    between_df.to_excel(writer, sheet_name="between_raw", index=False)

import re, numpy as np, pandas as pd
from scipy.stats import mannwhitneyu, wilcoxon
from statsmodels.stats.multitest import multipletests
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.formatting.rule import CellIsRule
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.shared import Pt
from docx.oxml.ns import qn

# ===== 1) Загрузка =====
try:
    from google.colab import files  # type: ignore
    uploaded = files.upload()       # выберите .xlsx
    infile = next(iter(uploaded.keys()))
except Exception:
    infile = "/content/файл_для_расчета.xlsx"

df = pd.read_excel(infile)

# ===== 2) Настройки =====
GROUP_COL_CANDS = ["диагноз", "диагнoз", "group", "Группа", "группа"]
group_col = next((c for c in GROUP_COL_CANDS if c in df.columns), None)
if group_col is None:
    raise RuntimeError("Добавьте столбец 'диагноз' (1=ИЗЛ, 2=ХОБЛ).")

GROUP_LABELS = {1: "ИЗЛ", 2: "ХОБЛ"}
DEC = 3

year_re = re.compile(r"_(\d{4})$")
marker_cols = [c for c in df.columns if year_re.search(str(c))]
if not marker_cols:
    raise RuntimeError("Нет столбцов вида '<показатель>_ГОД'.")

all_markers = sorted(set(re.sub(year_re, "", c) for c in marker_cols))
years = sorted(set(int(year_re.search(c).group(1)) for c in marker_cols))

# Нормализация: приводим к нижнему регистру, убираем _ и пробелы,
# заменяем кириллические «похожие» на латиницу (и наоборот не требуется).
def norm_text(s: str) -> str:
    m = {"С":"C","с":"c","О":"O","о":"o","Т":"T","т":"t","М":"M","м":"m",
         "Х":"X","х":"x","В":"B","в":"b","Р":"P","р":"p","Н":"H","н":"h",
         "К":"K","к":"k","А":"A","а":"a","Е":"E","е":"e","У":"Y","у":"y",
         "Ё":"E","ё":"e","Й":"I","й":"i","Л":"L","л":"l","Д":"D","д":"d"}
    s2 = "".join(m.get(ch, ch) for ch in str(s))
    return s2.lower().replace("_", "").replace(" ", "")

# ---- РАСШИРЕННЫЙ словарь ключей пульмопрофиля (в «сыром» виде) ----
# ВАЖНО: мы нормализуем ключи той же функцией и сравниваем уже нормализованные строки.
pulmo_keys_raw = [
    # 6MWT, ЧД, разница
    "дистанция т6мх", "t6мх", "t6mx", "т6мх",
    "чд в минуту", "разница чдд т6мх",
    # SaO2, Борг, спирометрия
    "sao2 pct", "sao2", "шкала борга",
    "фжел", "офв1", "ofv1",
    # газы и лактат
    "co2 в выдыхаемом воздухе", "ph", "po2", "pco2", "lac",
    # >>> ПРОБЛЕМНЫЕ ИЗ СООБЩЕНИЯ:
    # объем лёгких — учитываем «обЪем/обЕм», a/о и т. п.
    "объем легких", "обьем легких", "обем легких",
    # денситометрические показатели — любые вариации «денсито… показат…»
    "денситометрические показатели", "денситометричеcкие показатели",
    "денситометричесkie показатели", "денсито",
    # плотность фиброза / общая «плотность»
    "плотность фиброза", "плотность"
]

# Нормализуем ключи и имена маркеров, потом сравниваем подстроки
keys_norm = [norm_text(k) for k in pulmo_keys_raw]
def is_pulmo_marker(name: str) -> bool:
    n = norm_text(name)
    return any(kn in n for kn in keys_norm)

pulmo_markers = [m for m in all_markers if is_pulmo_marker(m)]
# Если вдруг пусто — берём всё (но обычно это не понадобится)
if not pulmo_markers:
    pulmo_markers = all_markers.copy()

# ===== 3) LONG-таблица =====
rows = []
for m in pulmo_markers:
    for y in years:
        col = f"{m}_{y}"
        if col in df.columns:
            vals = pd.to_numeric(df[col], errors="coerce")
            rows.append(pd.DataFrame({
                "marker": m, "year": y,
                "group": df[group_col].map(GROUP_LABELS).fillna(df[group_col].astype(str)),
                "value": vals
            }))
long_df = pd.concat(rows, ignore_index=True)

def med_q1_q3(x):
    x = pd.to_numeric(pd.Series(x), errors="coerce").dropna()
    if len(x)==0: return np.nan, np.nan, np.nan, 0
    return float(np.nanmedian(x)), float(np.nanpercentile(x,25)), float(np.nanpercentile(x,75)), int(x.notna().sum())

def fnum(v):  return "NA" if pd.isna(v) else f"{v:.{DEC}g}"
def fstat(m,q1,q3,n,comma=False):
    t = (lambda s: s.replace(".", ",")) if comma else (lambda s: s)
    return f"{t(fnum(m))} [{t(fnum(q1))}; {t(fnum(q3))}] (n={n})"

# ===== 4) P-values =====
# Межгрупповые (по годам)
bt = []
for m in pulmo_markers:
    for y in years:
        sub = long_df[(long_df.marker==m)&(long_df.year==y)]
        v1 = sub[sub.group=="ИЗЛ"].value.dropna().values
        v2 = sub[sub.group=="ХОБЛ"].value.dropna().values
        p = mannwhitneyu(v1, v2, alternative="two-sided").pvalue if len(v1)>0 and len(v2)>0 else np.nan
        bt.append({"marker":m,"year":y,"p_between":p})
between_df = pd.DataFrame(bt)
if not between_df.empty:
    ms = between_df.p_between.notna()
    q = np.full(len(between_df), np.nan)
    if ms.any(): q[ms.values] = multipletests(between_df.loc[ms,"p_between"], method="fdr_bh")[1]
    between_df["q_between"] = q

# Внутригрупповые (только соседние годы)
pairs = {years[i+1]: years[i] for i in range(len(years)-1)}
wt = []
for m in pulmo_markers:
    for code,name in GROUP_LABELS.items():
        sub_idx = df[group_col]==code
        wide = {y: pd.to_numeric(df.loc[sub_idx, f"{m}_{y}"], errors="coerce")
                for y in years if f"{m}_{y}" in df.columns}
        if not wide: continue
        wide = pd.DataFrame(wide)
        for y2,y1 in pairs.items():
            if y2 in wide and y1 in wide:
                a,b = wide[y1],wide[y2]
                mask = a.notna() & b.notna()
                p = wilcoxon(a[mask], b[mask], zero_method="wilcox").pvalue if mask.sum()>1 else np.nan
                wt.append({"marker":m,"group":name,"year":y2,"prev_year":y1,"p_within":p})
within_df = pd.DataFrame(wt)
if not within_df.empty:
    ms = within_df.p_within.notna()
    q = np.full(len(within_df), np.nan)
    if ms.any(): q[ms.values] = multipletests(within_df.loc[ms,"p_within"], method="fdr_bh")[1]
    within_df["q_within"] = q

# ===== 5) Сборка «страницы» =====
label = {y:(["a)","b)","c)","d)"][i] if i<4 else f"{i+1})") for i,y in enumerate(years)}
rows = []
for m in pulmo_markers:
    for y in years:
        sub = long_df[(long_df.marker==m)&(long_df.year==y)]
        mi,q1i,q3i,ni = med_q1_q3(sub[sub.group=="ИЗЛ"].value)
        mc,q1c,q3c,nc = med_q1_q3(sub[sub.group=="ХОБЛ"].value)
        rb = between_df[(between_df.marker==m)&(between_df.year==y)]
        pb,qb = (rb.p_between.item(), rb.q_between.item()) if not rb.empty else (np.nan,np.nan)
        rwi = within_df[(within_df.marker==m)&(within_df.group=="ИЗЛ")&(within_df.year==y)]
        pwi,qwi = (rwi.p_within.item(), rwi.q_within.item()) if not rwi.empty else (np.nan,np.nan)
        rwc = within_df[(within_df.marker==m)&(within_df.group=="ХОБЛ")&(within_df.year==y)]
        pwc,qwc = (rwc.p_within.item(), rwc.q_within.item()) if not rwc.empty else (np.nan,np.nan)
        rows.append({
            "Variables": m,
            "строка": f"{label[y]} {y} год",
            "МЕ [Q1;Q3] ИЗЛ": fstat(mi,q1i,q3i,ni),
            "Р внутригрупповой (ИЗЛ)": (np.nan if y==years[0] else pwi),
            "Р скорректированный (ИЗЛ)": (np.nan if y==years[0] else qwi),
            "МЕ [Q1;Q3] ХОБЛ": fstat(mc,q1c,q3c,nc),
            "Р внутригрупповой (ХОБЛ)": (np.nan if y==years[0] else pwc),
            "Р скорректированный (ХОБЛ)": (np.nan if y==years[0] else qwc),
            "Р межгрупповой": pb,
            "Р скорректированный (межгрупповой)": qb
        })
pulmo_page = pd.DataFrame(rows)
pulmo_page["year"] = pulmo_page["строка"].str.extract(r"(\d{4})").astype(int)
pulmo_page = pulmo_page.sort_values(["Variables","year"]).drop(columns=["year"]).reset_index(drop=True)

# ===== 6) Excel (оформление) =====
out_xlsx = "pulmo_profile_formatted.xlsx"
out_xlsx = "pulmo_profile_formatted.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    pulmo_page.to_excel(writer, sheet_name="pulmo_profile", index=False)
    within_df.to_excel(writer, sheet_name="within_raw", index=False)
    between_df.to_excel(writer, sheet_name="between_raw", index=False)

wb = load_workbook(out_xlsx); ws = wb["pulmo_profile"]
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4F81BD")
center = Alignment(horizontal="center", vertical="center", wrap_text=True)
left   = Alignment(horizontal="left",   vertical="center", wrap_text=True)
thin   = Side(border_style="thin", color="D9D9D9")
border = Border(left=thin, right=thin, top=thin, bottom=thin)
widths = {1:34,2:12,3:22,4:14,5:16,6:22,7:14,8:16,9:14,10:16}
for i,w in widths.items(): ws.column_dimensions[chr(ord('A')+i-1)].width = w
for c in ws[1]: c.font=header_font; c.fill=header_fill; c.alignment=center; c.border=border
ws.freeze_panes="A2"
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for j,cell in enumerate(row, start=1):
        cell.alignment = left if j in (1,2,3,6) else center
        cell.border = border
for col in [4,5,7,8,9,10]:
    L = chr(ord('A')+col-1)
    for r in range(2, ws.max_row+1):
        cell = ws.cell(row=r, column=col)
        if isinstance(cell.value,float) and np.isnan(cell.value): cell.value=None
        cell.number_format="0.0000"
    for color,thr in [("FFF2CC",0.05),("FCE4D6",0.01),("F8CBAD",0.001)]:
        ws.conditional_formatting.add(f"{L}2:{L}{ws.max_row}",
            CellIsRule(operator='lessThan', formula=[str(thr)],
                       fill=PatternFill("solid", fgColor=color)))
wb.save(out_xlsx)

# ===== 7) Word (таблица) =====
doc = Document()
style = doc.styles['Normal']; style.font.name='Times New Roman'
style._element.rPr.rFonts.set(qn('w:eastAsia'),'Times New Roman'); style.font.size=Pt(11)
p = doc.add_paragraph("Пульмонологический профиль"); p.alignment = WD_ALIGN_PARAGRAPH.CENTER; p.runs[0].bold=True
n_ild, n_copd = int((df[group_col]==1).sum()), int((df[group_col]==2).sum())
doc.add_paragraph(f"ИЗЛ: n={n_ild};  ХОБЛ: n={n_copd}")
headers = ["Variables","Год","МЕ [Q1;Q3] ИЗЛ","p внутригр. (ИЗЛ)","q (ИЗЛ)",
           "МЕ [Q1;Q3] ХОБЛ","p внутригр. (ХОБЛ)","q (ХОБЛ)","p межгр.","q межгр."]
tbl = doc.add_table(rows=1, cols=len(headers)); tbl.alignment=WD_TABLE_ALIGNMENT.CENTER; tbl.style='Table Grid'
for i,h in enumerate(headers): tbl.rows[0].cells[i].text = h
for m in pulmo_page["Variables"].unique():
    block = pulmo_page[pulmo_page["Variables"]==m]
    start = len(tbl.rows)
    for _,row in block.iterrows():
        tr = tbl.add_row().cells
        tr[0].text = m
        tr[1].text = row["строка"]
        tr[2].text = str(row["МЕ [Q1;Q3] ИЗЛ"]).replace(".", ",")
        tr[3].text = "" if pd.isna(row["Р внутригрупповой (ИЗЛ)"]) else f"{row['Р внутригрупповой (ИЗЛ)']:.4f}".replace(".", ",")
        tr[4].text = "" if pd.isna(row["Р скорректированный (ИЗЛ)"]) else f"{row['Р скорректированный (ИЗЛ)']:.4f}".replace(".", ",")
        tr[5].text = str(row["МЕ [Q1;Q3] ХОБЛ"]).replace(".", ",")
        tr[6].text = "" if pd.isna(row["Р внутригрупповой (ХОБЛ)"]) else f"{row['Р внутригрупповой (ХОБЛ)']:.4f}".replace(".", ",")
        tr[7].text = "" if pd.isna(row["Р скорректированный (ХОБЛ)"]) else f"{row['Р скорректированный (ХОБЛ)']:.4f}".replace(".", ",")
        tr[8].text = "" if pd.isna(row["Р межгрупповой"]) else f"{row['Р межгрупповой']:.4f}".replace(".", ",")
        tr[9].text = "" if pd.isna(row["Р скорректированный (межгрупповой)"]) else f"{row['Р скорректированный (межгрупповой)']:.4f}".replace(".", ",")
    end = len(tbl.rows)-1
    if end >= start:
        tbl.cell(start,0).merge(tbl.cell(end,0)).text = m
doc.add_paragraph("Примечание: Wilcoxon — соседние годы (2024 vs 2023; 2025 vs 2024); Mann–Whitney — по годам; q — FDR (BH).")
out_xlsx, out_docx = "pulmo_profile_formatted.xlsx", "pulmo_profile_table.docx"
doc.save(out_docx)

print("Готово:", out_xlsx, "и", out_docx)
try:
    from google.colab import files  # type: ignore
    files.download(out_xlsx); files.download(out_docx)
except Exception:
    pass


Saving pulmo_polarity_aligned итоговый файл.xlsx to pulmo_polarity_aligned итоговый файл (15).xlsx
Готово: pulmo_profile_formatted.xlsx и pulmo_profile_table.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Новый раздел